## Setup

In [1]:
import os
from dotenv import load_dotenv
from src import util
from statsmodels.graphics.tsaplots import month_plot, quarter_plot
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose


import xgboost as xgb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix


In [2]:
load_dotenv()

slided_dfs_path = os.getenv("SLIDED_DFS_CSV_PATH")

slided_data = pd.read_parquet(os.path.join(slided_dfs_path, "data_slided_V2.parquet"))

In [3]:
slided_data

xs_mean_1h     xs_std_1h     xs_max_1h    xs_mean_6h  \
ds                                                                            
1996-01-01 00:01:00  9.110000e-10  1.979899e-11  9.250000e-10  9.110000e-10   
1996-01-01 00:02:00  9.163333e-10  1.677299e-11  9.270000e-10  9.163333e-10   
1996-01-01 00:03:00  9.172500e-10  1.381726e-11  9.270000e-10  9.172500e-10   
1996-01-01 00:04:00  9.204000e-10  1.388524e-11  9.330000e-10  9.204000e-10   
1996-01-01 00:05:00  9.193333e-10  1.269120e-11  9.330000e-10  9.193333e-10   
...                           ...           ...           ...           ...   
2024-12-31 23:54:00  2.435012e-07  1.461810e-07  1.007250e-06  5.357717e-07   
2024-12-31 23:55:00  2.305124e-07  1.063709e-07  7.818082e-07  5.363405e-07   
2024-12-31 23:56:00  2.212888e-07  7.795530e-08  6.006925e-07  5.369138e-07   
2024-12-31 23:57:00  2.151898e-07  6.002076e-08  4.691412e-07  5.375014e-07   
2024-12-31 23:58:00  2.114305e-07  5.008701e-08  3.773774e-07  5.381131e-07   

                        xs_std_6h     xs_max_6h   xs_mean_12h    xs_std_12h  \
ds                                                                            
1996-01-01 00:01:00  1.979899e-11  9.250000e-10  9.110000e-10  1.979899e-11   
1996-01-01 00:02:00  1.677299e-11  9.270000e-10  9.163333e-10  1.677299e-11   
1996-01-01 00:03:00  1.381726e-11  9.270000e-10  9.172500e-10  1.381726e-11   
1996-01-01 00:04:00  1.388524e-11  9.330000e-10  9.204000e-10  1.388524e-11   
1996-01-01 00:05:00  1.269120e-11  9.330000e-10  9.193333e-10  1.269120e-11   
...                           ...           ...           ...           ...   
2024-12-31 23:54:00  8.248068e-07  3.523976e-06  3.058905e-07  6.274768e-07   
2024-12-31 23:55:00  8.245229e-07  3.523976e-06  3.060488e-07  6.274427e-07   
2024-12-31 23:56:00  8.242359e-07  3.523976e-06  3.062156e-07  6.274060e-07   
2024-12-31 23:57:00  8.239441e-07  3.523976e-06  3.063945e-07  6.273672e-07   
2024-12-31 23:58:00  8.236430e-07  3.523976e-06  3.065916e-07  6.273251e-07   

                       xs_max_12h   xs_mean_24h  ...    xl_std_30D  \
ds                                               ...                 
1996-01-01 00:01:00  9.250000e-10  9.110000e-10  ...  5.656854e-10   
1996-01-01 00:02:00  9.270000e-10  9.163333e-10  ...  4.618802e-10   
1996-01-01 00:03:00  9.270000e-10  9.172500e-10  ...  4.193249e-10   
1996-01-01 00:04:00  9.330000e-10  9.204000e-10  ...  5.674504e-10   
1996-01-01 00:05:00  9.330000e-10  9.193333e-10  ...  5.099020e-10   
...                           ...           ...  ...           ...   
2024-12-31 23:54:00  3.523976e-06  2.114836e-07  ...  5.722925e-06   
2024-12-31 23:55:00  3.523976e-06  2.115976e-07  ...  5.722929e-06   
2024-12-31 23:56:00  3.523976e-06  2.117168e-07  ...  5.722933e-06   
2024-12-31 23:57:00  3.523976e-06  2.118366e-07  ...  5.722938e-06   
2024-12-31 23:58:00  3.523976e-06  2.119606e-07  ...  5.722943e-06   

                       xl_max_30D  xl_deriv_5min  xl_deriv_15min  \
ds                                                                 
1996-01-01 00:01:00  1.460000e-08   8.000000e-10    8.000000e-10   
1996-01-01 00:02:00  1.460000e-08   0.000000e+00    0.000000e+00   
1996-01-01 00:03:00  1.460000e-08  -3.333333e-11   -3.333333e-11   
1996-01-01 00:04:00  1.460000e-08  -2.000000e-10   -2.000000e-10   
1996-01-01 00:05:00  1.460000e-08   2.000000e-11    2.000000e-11   
...                           ...            ...             ...   
2024-12-31 23:54:00  2.230703e-04  -3.748356e-08   -1.566180e-09   
2024-12-31 23:55:00  2.230703e-04  -2.949672e-08   -4.742867e-09   
2024-12-31 23:56:00  2.230703e-04  -2.189944e-08   -4.815847e-09   
2024-12-31 23:57:00  2.230703e-04  -4.218600e-09   -9.317800e-10   
2024-12-31 23:58:00  2.230703e-04   1.160090e-08    3.357833e-09   

                     xl_deriv_30min   xl_deriv_1h   xl_deriv_3h   xl_deriv_6h  \
ds                                                                              
1996-01-0

In [4]:
slided_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 14780343 entries, 1996-01-01 00:01:00 to 2024-12-31 23:58:00
Data columns (total 56 columns):
 #   Column               Dtype  
---  ------               -----  
 0   xs_mean_1h           float64
 1   xs_std_1h            float64
 2   xs_max_1h            float64
 3   xs_mean_6h           float64
 4   xs_std_6h            float64
 5   xs_max_6h            float64
 6   xs_mean_12h          float64
 7   xs_std_12h           float64
 8   xs_max_12h           float64
 9   xs_mean_24h          float64
 10  xs_std_24h           float64
 11  xs_max_24h           float64
 12  xs_mean_7D           float64
 13  xs_std_7D            float64
 14  xs_max_7D            float64
 15  xs_mean_14D          float64
 16  xs_std_14D           float64
 17  xs_max_14D           float64
 18  xs_mean_30D          float64
 19  xs_std_30D           float64
 20  xs_max_30D           float64
 21  xs_deriv_5min        float64
 22  xs_deriv_15min       float64
 23

In [5]:
wanted_cols = [c for c in slided_data.columns if c.startswith("xl_") or c == "target_class_in_24h"]
df_model_input = slided_data[wanted_cols]
df_model_input = df_model_input.resample('10min').last().ffill().dropna() #testar com .max()

In [6]:
df_model_input

xl_mean_1h     xl_std_1h     xl_max_1h    xl_mean_6h  \
ds                                                                            
1996-01-01 00:00:00  1.382000e-08  4.077036e-10  1.460000e-08  1.382000e-08   
1996-01-01 00:10:00  1.386000e-08  3.662039e-10  1.460000e-08  1.386000e-08   
1996-01-01 00:20:00  1.465000e-08  1.199353e-09  1.690000e-08  1.465000e-08   
1996-01-01 00:30:00  1.500750e-08  1.278498e-09  1.760000e-08  1.500750e-08   
1996-01-01 00:40:00  1.493600e-08  1.207757e-09  1.760000e-08  1.493600e-08   
...                           ...           ...           ...           ...   
2024-12-31 23:10:00  1.741547e-05  8.155662e-06  2.906757e-05  8.042177e-06   
2024-12-31 23:20:00  1.386945e-05  8.019732e-06  2.906757e-05  8.102606e-06   
2024-12-31 23:30:00  1.089013e-05  7.042411e-06  2.906757e-05  8.168346e-06   
2024-12-31 23:40:00  8.751141e-06  5.947402e-06  2.906757e-05  8.258113e-06   
2024-12-31 23:50:00  6.453968e-06  1.472110e-06  1.231739e-05  8.341436e-06   

                        xl_std_6h     xl_max_6h   xl_mean_12h    xl_std_12h  \
ds                                                                            
1996-01-01 00:00:00  4.077036e-10  1.460000e-08  1.382000e-08  4.077036e-10   
1996-01-01 00:10:00  3.662039e-10  1.460000e-08  1.386000e-08  3.662039e-10   
1996-01-01 00:20:00  1.199353e-09  1.690000e-08  1.465000e-08  1.199353e-09   
1996-01-01 00:30:00  1.278498e-09  1.760000e-08  1.500750e-08  1.278498e-09   
1996-01-01 00:40:00  1.207757e-09  1.760000e-08  1.493600e-08  1.207757e-09   
...                           ...           ...           ...           ...   
2024-12-31 23:10:00  7.965353e-06  2.906757e-05  5.907889e-06  6.032180e-06   
2024-12-31 23:20:00  7.935948e-06  2.906757e-05  5.934341e-06  6.025285e-06   
2024-12-31 23:30:00  7.906699e-06  2.906757e-05  5.967761e-06  6.017797e-06   
2024-12-31 23:40:00  7.865352e-06  2.906757e-05  6.002706e-06  6.011759e-06   
2024-12-31 23:50:00  7.824673e-06  2.906757e-05  6.028887e-06  6.007762e-06   

                       xl_max_12h   xl_mean_24h  ...   xl_mean_30D  \
ds                                               ...                 
1996-01-01 00:00:00  1.460000e-08  1.382000e-08  ...  1.382000e-08   
1996-01-01 00:10:00  1.460000e-08  1.386000e-08  ...  1.386000e-08   
1996-01-01 00:20:00  1.690000e-08  1.465000e-08  ...  1.465000e-08   
1996-01-01 00:30:00  1.760000e-08  1.500750e-08  ...  1.500750e-08   
1996-01-01 00:40:00  1.760000e-08  1.493600e-08  ...  1.493600e-08   
...                           ...           ...  ...           ...   
2024-12-31 23:10:00  2.906757e-05  5.112266e-06  ...  3.554018e-06   
2024-12-31 23:20:00  2.906757e-05  5.111699e-06  ...  3.554895e-06   
2024-12-31 23:30:00  2.906757e-05  5.114725e-06  ...  3.555906e-06   
2024-12-31 23:40:00  2.906757e-05  5.126298e-06  ...  3.557003e-06   
2024-12-31 23:50:00  2.906757e-05  5.140818e-06  ...  3.557983e-06   

                       xl_std_30D    xl_max_30D  xl_deriv_5min  \
ds                                                               
1996-01-01 00:00:00  4.077036e-10  1.460000e-08   1.200000e-10   
1996-01-01 00:10:00  3.662039e-10  1.460000e-08   6.000000e-11   
1996-01-01 00:20:00  1.199353e-09  1.690000e-08  -2.000000e-11   
1996-01-01 00:30:00  1.278498e-09  1.760000e-08  -4.800000e-10   
1996-01-01 00:40:00  1.207757e-09  1.760000e-08  -4.200000e-10   
...                           ...           ...            ...   
2024-12-31 23:10:00  5.722854e-06  2.230703e-04  -1.021528e-07   
2024-12-31 23:20:00  5.722833e-06  2.230703e-04   5.006494e-08   
2024-12-31 23:30:00  5.722846e-06  2.230703e-04   6.857236e-08   
2024-12-31 23:40:00  5.722901e-06  2.230703e-04   1.656350e-08   
2024-12-31 23:50:00  5.722943e-06  2.230703e-04   1.160090e-08   

                     xl_deriv_15min  xl_deriv_30min   xl_deriv_1h  \
ds                                                                  
1996-01-01 00:00:00   -2.222222e-11   -2.222222e-11 -2.2222

## Preparing Data

In [7]:
y = df_model_input["target_class_in_24h"].apply(lambda lb: 1 if lb>0 else 0)
x = df_model_input.drop(columns=["target_class_in_24h"])

In [8]:
train_pct = 0.7
val_pct = (1-train_pct)/2
test_pct = (1-train_pct)/2

n = len(x)
train_end = int(n*train_pct)
val_end = int(n * (val_pct+train_pct) )

x_train = x.iloc[:train_end]
y_train = y.iloc[:train_end]

x_val = x.iloc[train_end:val_end]
y_val = y.iloc[train_end:val_end]

x_test = x.iloc[val_end:]
y_test = y.iloc[val_end:]

In [9]:
x_train

xl_mean_1h     xl_std_1h     xl_max_1h    xl_mean_6h  \
ds                                                                            
1996-01-01 00:00:00  1.382000e-08  4.077036e-10  1.460000e-08  1.382000e-08   
1996-01-01 00:10:00  1.386000e-08  3.662039e-10  1.460000e-08  1.386000e-08   
1996-01-01 00:20:00  1.465000e-08  1.199353e-09  1.690000e-08  1.465000e-08   
1996-01-01 00:30:00  1.500750e-08  1.278498e-09  1.760000e-08  1.500750e-08   
1996-01-01 00:40:00  1.493600e-08  1.207757e-09  1.760000e-08  1.493600e-08   
...                           ...           ...           ...           ...   
2016-04-20 01:30:00  2.254173e-07  7.551093e-09  2.451500e-07  2.940705e-07   
2016-04-20 01:40:00  2.260857e-07  8.144930e-09  2.451500e-07  2.910954e-07   
2016-04-20 01:50:00  2.288933e-07  9.581287e-09  2.533900e-07  2.899577e-07   
2016-04-20 02:00:00  2.334470e-07  1.363193e-08  2.894400e-07  2.897709e-07   
2016-04-20 02:10:00  2.553492e-07  4.729392e-08  3.906100e-07  2.930129e-07   

                        xl_std_6h     xl_max_6h   xl_mean_12h    xl_std_12h  \
ds                                                                            
1996-01-01 00:00:00  4.077036e-10  1.460000e-08  1.382000e-08  4.077036e-10   
1996-01-01 00:10:00  3.662039e-10  1.460000e-08  1.386000e-08  3.662039e-10   
1996-01-01 00:20:00  1.199353e-09  1.690000e-08  1.465000e-08  1.199353e-09   
1996-01-01 00:30:00  1.278498e-09  1.760000e-08  1.500750e-08  1.278498e-09   
1996-01-01 00:40:00  1.207757e-09  1.760000e-08  1.493600e-08  1.207757e-09   
...                           ...           ...           ...           ...   
2016-04-20 01:30:00  1.496797e-07  1.076700e-06  2.833159e-07  1.121442e-07   
2016-04-20 01:40:00  1.498156e-07  1.076700e-06  2.821563e-07  1.122764e-07   
2016-04-20 01:50:00  1.500448e-07  1.076700e-06  2.806384e-07  1.121014e-07   
2016-04-20 02:00:00  1.501337e-07  1.076700e-06  2.787391e-07  1.115226e-07   
2016-04-20 02:10:00  1.502794e-07  1.076700e-06  2.785514e-07  1.114236e-07   

                       xl_max_12h   xl_mean_24h  ...    xl_max_14D  \
ds                                               ...                 
1996-01-01 00:00:00  1.460000e-08  1.382000e-08  ...  1.460000e-08   
1996-01-01 00:10:00  1.460000e-08  1.386000e-08  ...  1.460000e-08   
1996-01-01 00:20:00  1.690000e-08  1.465000e-08  ...  1.690000e-08   
1996-01-01 00:30:00  1.760000e-08  1.500750e-08  ...  1.760000e-08   
1996-01-01 00:40:00  1.760000e-08  1.493600e-08  ...  1.760000e-08   
...                           ...           ...  ...           ...   
2016-04-20 01:30:00  1.076700e-06  2.834883e-07  ...  6.717000e-05   
2016-04-20 01:40:00  1.076700e-06  2.832209e-07  ...  6.717000e-05   
2016-04-20 01:50:00  1.076700e-06  2.830763e-07  ...  6.717000e-05   
2016-04-20 02:00:00  1.076700e-06  2.827832e-07  ...  6.717000e-05   
2016-04-20 02:10:00  1.076700e-06  2.830734e-07  ...  6.717000e-05   

                      xl_mean_30D    xl_std_30D    xl_max_30D  xl_deriv_5min  \
ds                                                                             
1996-01-01 00:00:00  1.382000e-08  4.077036e-10  1.460000e-08   1.200000e-10   
1996-01-01 00:10:00  1.386000e-08  3.662039e-10  1.460000e-08   6.000000e-11   
1996-01-01 00:20:00  1.465000e-08  1.199353e-09  1.690000e-08  -2.000000e-11   
1996-01-01 00:30:00  1.500750e-08  1.278498e-09  1.760000e-08  -4.800000e-10   
1996-01-01 00:40:00  1.493600e-08  1.207757e-09  1.760000e-08  -4.200000e-10   
...                           ...           ...           ...            ...   
2016-04-20 01:30:00  3.201666e-07  1.092767e-06  6.717000e-05   3.400000e-10   
2016-04-20 01:40:00  3.201849e-07  1.092765e-06  6.717000e-05   4.572000e-09   
2016-04-20 01:50:00  3.202044e-07  1.092763e-06  6.717000e-05  -4.506000e-09   
2016-04-20 02:00:00  3.202253e-07  1.092760e-06  6.717000e-05   1.144400e-08   
2016-04-20 02:10:00  3.202722e-07  1.092758e-06  6.717000e-05  -1.161400e-08   

               

In [12]:
y_train

ds
1996-01-01 00:00:00    0
1996-01-01 00:10:00    0
1996-01-01 00:20:00    0
1996-01-01 00:30:00    0
1996-01-01 00:40:00    0
                      ..
2016-04-20 01:30:00    1
2016-04-20 01:40:00    1
2016-04-20 01:50:00    1
2016-04-20 02:00:00    1
2016-04-20 02:10:00    1
Freq: 10min, Name: target_class_in_24h, Length: 1067774, dtype: int64

In [23]:
y_val

ds
2016-04-20 02:20:00    1
2016-04-20 02:30:00    1
2016-04-20 02:40:00    1
2016-04-20 02:50:00    1
2016-04-20 03:00:00    1
                      ..
2020-08-26 00:20:00    0
2020-08-26 00:30:00    0
2020-08-26 00:40:00    0
2020-08-26 00:50:00    0
2020-08-26 01:00:00    0
Freq: 10min, Name: target_class_in_24h, Length: 228809, dtype: int64

In [26]:
y_val

ds
2016-04-20 02:20:00    1
2016-04-20 02:30:00    1
2016-04-20 02:40:00    1
2016-04-20 02:50:00    1
2016-04-20 03:00:00    1
                      ..
2020-08-26 00:20:00    0
2020-08-26 00:30:00    0
2020-08-26 00:40:00    0
2020-08-26 00:50:00    0
2020-08-26 01:00:00    0
Freq: 10min, Name: target_class_in_24h, Length: 228809, dtype: int64

## Training and Testing Dataframes

In [ ]:
training_percentage = 80
all_years = list(range(begin_year, end_year + 1))
total_years = len(all_years)

split_index = int(total_years * (training_percentage / 100))

train_years = all_years[:split_index]
test_years = all_years[split_index:]

print(f"Anos de Treino ({len(train_years)}): {train_years[0]} até {train_years[-1]}")
print(f"Anos de Teste ({len(test_years)}): {test_years[0]} até {test_years[-1]}")

def create_dataset(years_list):
    xr_list = []
    xr_mean_list = []
    xr_max_list = []

    for y_ in years_list:
        xr_list.append(data_slided[y_]['xr'])
        xr_mean_list.append(data_slided[y_]['xr_mean'])
        xr_max_list.append(data_slided[y_]['xr_max'])

    return {
        'xr': pd.concat(xr_list),
        'xr_mean': pd.concat(xr_mean_list),
        'xr_max': pd.concat(xr_max_list),
    }

training_dfs = create_dataset(train_years)
testing_dfs = create_dataset(test_years)

In [ ]:
flare_map = {'No Flare': 0, 'A': 1, 'B': 2, 'C': 3, 'M': 4, 'X': 5}

for key in training_dfs.keys():
    training_dfs[key] = training_dfs[key].rename(columns={'flare_class': 'y'})
    training_dfs[key]['y'] = training_dfs[key]['y'].map(flare_map)

In [ ]:
training_dfs['xr']

## Exploratory Data Analysis

In [ ]:
eda_training = training_dfs['xr']['xr_lag_1_min_1']
fig = plt.figure(figsize=(16,9))
eda_training.plot()

In [ ]:
decomposition = seasonal_decompose(eda_training, model='mul',period=481800)
fig = decomposition.plot()
fig.set_size_inches(16,9)
plt.show()

In [ ]:
fig = month_plot(eda_train.resample('ME').mean())
fig.set_size_inches(16,9)
plt.show()

## A

In [ ]:
data_slided[2020]['xr'].head().to_csv()

In [ ]:
data_slided[2023]['xr_mean']

In [ ]:
data_slided[2001]['xr_max']